In [188]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor, XGBClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score





In [172]:
# Đọc dữ liệu
data = pd.read_csv('./clean_big_data.csv')
data['DT'] = pd.to_datetime(data['DT'], unit='s')
data = data.sort_values('DT')

In [173]:
def create_features(df, label, hours):
    df_new = df.copy()
    for hour in range(1, hours + 1):
        df_new[f'Temp_-{hour}h'] = df_new['Temp'].shift(hour)
        df_new[f'Humidity_-{hour}h'] = df_new['Humidity'].shift(hour)

    df_new['Hour'] = df_new['DT'].dt.hour
    df_new['DayOfWeek'] = df_new['DT'].dt.dayofweek

    df_new = df_new.dropna()
    X = df_new[[f'Temp_-{h}h' for h in range(1, hours + 1)] +
                [f'Humidity_-{h}h' for h in range(1, hours + 1)] +
                ['Hour', 'DayOfWeek']]
    y = df_new[label]
    return X, y

In [174]:
hours_past = 4
X_rain, y_rain = create_features(data, 'Is Rain', hours_past)
X_temp, y_temp = create_features(data, 'Temp', hours_past)
X_humidity, y_humidity = create_features(data, 'Humidity', hours_past)

---

In [175]:
# Tạo các đặc trưng từ dữ liệu quá khứ
def create_features(df, label, hours):
    df_new = df.copy()
    for hour in range(1, hours + 1):
        df_new[f'Temp_-{hour}h'] = df_new['Temp'].shift(hour)
        df_new[f'Humidity_-{hour}h'] = df_new['Humidity'].shift(hour)

    df_new['Hour'] = df_new['DT'].dt.hour
    df_new['DayOfWeek'] = df_new['DT'].dt.dayofweek

    df_new = df_new.dropna()
    X = df_new[[f'Temp_-{h}h' for h in range(1, hours + 1)] +
                [f'Humidity_-{h}h' for h in range(1, hours + 1)] +
                ['Hour', 'DayOfWeek']]
    y = df_new[label]
    return X, y

In [176]:
# Tạo đặc trưng cho dự đoán xác suất mưa, nhiệt độ và độ ẩm
hours_past = 5
X_rain, y_rain = create_features(data, 'Is Rain', hours_past)
X_temp, y_temp = create_features(data, 'Temp', hours_past)
X_humidity, y_humidity = create_features(data, 'Humidity', hours_past)

In [177]:
# Chia dữ liệu
X_train_rain, X_test_rain, y_train_rain, y_test_rain = train_test_split(X_rain, y_rain, test_size=0.2, random_state=42)
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)
X_train_humidity, X_test_humidity, y_train_humidity, y_test_humidity = train_test_split(X_humidity, y_humidity, test_size=0.2, random_state=42)

In [178]:
scaler = StandardScaler()
X_train_rain_scaled = scaler.fit_transform(X_train_rain)
X_test_rain_scaled = scaler.transform(X_test_rain)
X_train_temp_scaled = scaler.fit_transform(X_train_temp)
X_test_temp_scaled = scaler.transform(X_test_temp)
X_train_humidity_scaled = scaler.fit_transform(X_train_humidity)
X_test_humidity_scaled = scaler.transform(X_test_humidity)

In [179]:
model_temp = LinearRegression()
model_temp.fit(X_train_temp_scaled, y_train_temp)

LinearRegression()

In [180]:
# Huấn luyện mô hình dự đoán độ ẩm
model_humidity = LinearRegression()
model_humidity.fit(X_train_humidity_scaled, y_train_humidity)

LinearRegression()

In [189]:
model_results = {}
models = {
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "XGBoost": XGBClassifier(),
    "Logistic Regression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "Gradient": GradientBoostingClassifier(),
    "DecisionTree": DecisionTreeClassifier()
}
# Thực hiện cross-validation cho từng model
for model_name, model in models.items():
    scores = cross_val_score(model, X_train_rain_scaled, y_train_rain, cv=5, scoring='accuracy')  # cv=5 dùng 5-fold CV
    model_results[model_name] = scores.mean()  # Lấy điểm trung bình

# In ra kết quả
for model_name, accuracy in model_results.items():
    print(f"{model_name}: {accuracy:.4f}")

Random Forest: 0.8407
SVM: 0.7886
XGBoost: 0.8390
Logistic Regression: 0.7287
KNN: 0.8092
Gradient: 0.7958
DecisionTree: 0.7773


In [191]:

param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_rain_scaled, y_train_rain)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

Best parameters: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best accuracy: 0.8429211430437347


In [187]:

param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.1, 0.05],
    'max_depth': [4, 6, 8],
    'subsample': [0.7, 0.8, 1.0]
}


grid_search = GridSearchCV(XGBClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_rain_scaled, y_train_rain)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

Best parameters: {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 300, 'subsample': 0.8}
Best accuracy: 0.8432960022369569


In [181]:
# model_rain = RandomForestClassifier()
# model_rain = GaussianNB()
# model_rain = GradientBoostingClassifier()
# model_rain = KNeighborsClassifier()
# model_rain = DecisionTreeClassifier()
model_rain = XGBClassifier(
        learning_rate=0.01, 
        n_estimators=500,
        subsample=0.8,
        gamma=0.1
    )
model_rain.fit(X_train_rain_scaled, y_train_rain)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [182]:

# Dự đoán nhiệt độ trên tập kiểm tra
y_pred_temp = model_temp.predict(X_test_temp_scaled)
mse_temp = mean_squared_error(y_test_temp, y_pred_temp)
print("MSE (Temp):", mse_temp)

# Dự đoán độ ẩm trên tập kiểm tra
y_pred_humidity = model_humidity.predict(X_test_humidity_scaled)
mse_humidity = mean_squared_error(y_test_humidity, y_pred_humidity)
print("MSE (Humidity):", mse_humidity)

# Dự đoán có mưa hay không
y_pred_rain = model_rain.predict(X_test_rain_scaled)
accuracy_rain = accuracy_score(y_test_rain, y_pred_rain)
print("Accuracy (Is Rain):", accuracy_rain)


MSE (Temp): 1.5818074064234708
MSE (Humidity): 36.33387536821735
Accuracy (Is Rain): 0.8254127063531765


In [183]:
# Hàm dự đoán cho nhiều giờ tiếp theo
def predict_weather_multiple(temp_history, humidity_history, hours_ahead):
    if len(temp_history) != hours_past or len(humidity_history) != hours_past:
        raise ValueError(f"Cần cung cấp {hours_past} giá trị nhiệt độ và độ ẩm gần nhất")

    predictions = []

    for _ in range(hours_ahead):
        input_data = np.array(temp_history + humidity_history + [0, 0]).reshape(1, -1)
        input_data_scaled_rain = scaler.transform(input_data)
        input_data_scaled_temp = scaler.transform(input_data)
        input_data_scaled_humidity = scaler.transform(input_data)

        rain_prob = model_rain.predict_proba(input_data_scaled_rain)[0][1]
        temp_forecast = model_temp.predict(input_data_scaled_temp)[0]
        humidity_forecast = model_humidity.predict(input_data_scaled_humidity)[0]

        predictions.append((temp_forecast, humidity_forecast, rain_prob))

        temp_history.append(temp_forecast)
        humidity_history.append(humidity_forecast)
        temp_history.pop(0)
        humidity_history.pop(0)

    return predictions

In [184]:
# Ví dụ sử dụng
temp_history = [28.62, 30.98, 28.56, 27,28]
humidity_history = [76, 79, 80, 78,80]
hours_ahead = 8

predictions = predict_weather_multiple(temp_history, humidity_history, hours_ahead)

print(f"\nDự đoán thời tiết cho {hours_ahead} giờ tiếp theo:")
for i, (temp, humidity, rain_prob) in enumerate(predictions):
    print(f"Giờ {i + 1}: Nhiệt độ dự đoán: {temp:.1f}°C, Độ ẩm dự đoán: {humidity:.1f}%, Xác suất mưa: {rain_prob:.2%}")



Dự đoán thời tiết cho 8 giờ tiếp theo:
Giờ 1: Nhiệt độ dự đoán: 29.5°C, Độ ẩm dự đoán: 75.3%, Xác suất mưa: 42.48%
Giờ 2: Nhiệt độ dự đoán: 30.9°C, Độ ẩm dự đoán: 71.9%, Xác suất mưa: 40.50%
Giờ 3: Nhiệt độ dự đoán: 29.0°C, Độ ẩm dự đoán: 75.6%, Xác suất mưa: 36.76%
Giờ 4: Nhiệt độ dự đoán: 27.9°C, Độ ẩm dự đoán: 78.9%, Xác suất mưa: 40.29%
Giờ 5: Nhiệt độ dự đoán: 28.7°C, Độ ẩm dự đoán: 78.0%, Xác suất mưa: 41.54%
Giờ 6: Nhiệt độ dự đoán: 30.2°C, Độ ẩm dự đoán: 72.6%, Xác suất mưa: 42.02%
Giờ 7: Nhiệt độ dự đoán: 31.1°C, Độ ẩm dự đoán: 68.9%, Xác suất mưa: 28.65%
Giờ 8: Nhiệt độ dự đoán: 29.5°C, Độ ẩm dự đoán: 73.8%, Xác suất mưa: 30.60%


c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScal